## Tradeoffs in Memory Size vs. Accesses to Parent Memories

This notebook will show you how to analyze tradeoffs in memory size versus accesses to
parent memories. The generated curve will show you, for a given memory size, the lower
bound of the number of accesses to parent memories.

This analysis is called Orojenesis, and it is introduced in "Mind the Gap: Attainable
Data Movement and Operational Intensity Bounds for Tensor Algorithms" by Qijing Huang,
Po-An Tsai, Joel S. Emer, Angshuman Parashar. The paper includes additional analysis and
example use cases for this tradeoff.

Our plan to do this analysis is the following:
- Set up a simple architecture with a main memory and a global buffer
- Tell the mapper to optimize for both main memory accesses and global buffer usage
- Observe the resulting Pareto frontier between global buffer usage (size) and the
  minimum number of accesses to main memory.

To this end, we have an "orojenesis" architecture in the examples/arches directory.
Let's take a look at it:


In [ ]:
from IPython.display import Markdown, display
import accelforge as af


display(Markdown(f"""
``` yaml
{open(af.examples.arches.orojenesis).read()}
```
"""))


First import AccelForge and initialize a Spec. We'll use a 4096x4096x4096 matrix
multiply as our workload.

In [ ]:
import accelforge as af
from pathlib import Path

examples_dir = Path("../../examples")

spec = af.Spec.from_yaml(
    af.examples.arches.orojenesis,
    af.examples.workloads.matmuls,
    jinja_parse_data={"N_EINSUMS": 1, "M": 4096, "KN": 4096},
)
spec.mapper.metrics = af.Metrics.ENERGY | af.Metrics.RESOURCE_USAGE
results = spec.map_workload_to_arch()

Let's plot the results. We can see that, on a log-log scale, the curve appears to be
linear.

In [ ]:
import matplotlib.pyplot as plt

results.data.sort_values("Total<SEP>energy", inplace=True)
fig, ax = plt.subplots(figsize=(10, 10))
ax.plot(
    [x * spec.arch.find("GlobalBuffer").size for x in results.resource_usage()["GlobalBuffer"]],
    results.energy(),
    marker="o",
    drawstyle="steps-pre"
)
ax.set_xlabel("On-Chip Memory Size (bits)")
ax.set_ylabel("Lowest-Attainable Off-Chip Accesses (bits)")
ax.set_xscale("log")
ax.set_yscale("log")
plt.show()

### Fused vs. Unfused

Let's also do a comparison of how the fusion affects this curve. We'll use the same
architecture, but this time with a larger workload. We'll run a GPT-3 6.7B attention
head with 8192 tokens.

Note: Depending on your machine, this may take 3-15 minutes to run.

In [ ]:
import accelforge as af
from pathlib import Path

examples_dir = Path("../../examples")

spec = af.Spec.from_yaml(
    af.examples.arches.orojenesis,
    af.examples.workloads.gpt3_6_7B,
    # af.examples.workloads.three_matmuls_annotated,
    jinja_parse_data={"N_TOKENS": 8192}
)
spec.mapper.metrics = af.Metrics.ENERGY | af.Metrics.RESOURCE_USAGE

# FUSED
spec.arch.find("MainMemory").tensors.keep = "~Intermediates"
spec.arch.find("MainMemory").tensors.may_keep = "All"
results_fused = spec.map_workload_to_arch()

# UNFUSED
spec.arch.find("MainMemory").tensors.keep = "All"
results_unfused = spec.map_workload_to_arch()

Let's plot the results. 


We can see that, for small on-chip memory sizes, there is an approximately-linear
relationship between on-chip memory size and the number of off-chip accesses.
Additionally in this region, fusion has negligible benefit. These are because the
on-chip memory is too small to fully reuse the tensors of any given Einsum, so off-chip
accesses are dominated by refetching tensors.

As we increase on-chip memory size, the curve begins to flatten out, and fusion begins
to have a larger effect. These are because the available on-chip memory is large enough
to store full tensors, which enables multiple fusion opportunities:

- Untiled fusion: We can generate a full output tensor, keep it on-chip, then use it for
  the next Einsum
- Tiled fusion: We can keep a full weight tensor on-chip, while consuming input tensor
  and sending output tensors one-tile-at-a-time from and to other Einsums without going
  off-chip.

Also note that fusion tends to be less helpful if we're refetching tensors multiple
times from off-chip, because fusion can save, at most, one fetch of a tensor per Einsum
(exchanging with another Einsum happens once). If we're refetching tensors multiple
times, we're likely better off using all available capacity to minimize refetches.

The curve slowly flattens out, rather than quickly, because each Einsum has a different
shape, and some Einsums enter the no-refetch, fusion-helpful region at smaller on-chip
memory sizes.

In [ ]:
results_unfused.resource_usage()
results_unfused.columns
import matplotlib.pyplot as plt
import numpy as np

fig, ax = plt.subplots(figsize=(10, 10))

results_fused.data.sort_values("Total<SEP>energy", inplace=True)
ax.plot(
    np.array(results_fused.resource_usage()["GlobalBuffer"]) * spec.arch.find("GlobalBuffer").size,
    results_fused.energy(),
    label="Fused",
    marker="o",
    drawstyle="steps-pre"
)

results_unfused.data.sort_values("Total<SEP>energy", inplace=True)
ax.plot(
    np.array(results_unfused.resource_usage()["GlobalBuffer"]) * spec.arch.find("GlobalBuffer").size,
    results_unfused.energy(),
    label="Unfused",
    marker="o",
    drawstyle="steps-pre"
)
ax.set_xlabel("On-Chip Memory Size (bits)")
ax.set_ylabel("Lowest-Attainable Off-Chip Accesses (bits)")
ax.set_xscale("log")
ax.set_yscale("log")
ax.legend()
plt.show()